In [ ]:
from imp import reload
import groundfloor as gf
reload(gf)

In [ ]:
funded = gf.GoundFloor('https://www.groundfloor.us/education/funded')
funded_df = funded.crawl()

In [ ]:
funded_df.columns

In [ ]:
funded_df.company.value_counts()

In [ ]:
funded_df[funded_df.borrower == 'John Mangham - principal'][['borrower', 'company', 'loan_status']]